In [ ]:
import random
import math
import gc
import sys
import numpy as np
import torch
import pickle
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
%matplotlib inline

sys.path.insert(0, '../../../methods')
from proposed import *

In [ ]:
# random seed
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# load data
data = np.load('../../../data\experiment_1/scenario_1/data_train.npy', allow_pickle=True)

In [ ]:
def get_coarray(X, sensor_pos, M):
    X_out = np.zeros((len(X), M-1, 2))
    X_mask = np.ones((len(X), M-1))

    for n in range(len(X)):
        #print(f"{n}/{len(X)}")
        counter = np.zeros(M-1)
        for i in range(sensor_pos[n].shape[0]):
            for j in range(i+1,sensor_pos[n].shape[0]):
                idx = int(abs(2*(sensor_pos[n][i]-sensor_pos[n][j]))-1)
                if sensor_pos[n][j] > sensor_pos[n][i]:
                    X_out[n,idx,0] = X_out[n,idx,0] + np.real(X[n][i,j])
                    X_out[n,idx,1] = X_out[n,idx,1] + np.imag(X[n][i,j])
                else:
                    X_out[n,idx,0] = X_out[n,idx,0] + np.real(X[n][j,i])
                    X_out[n,idx,1] = X_out[n,idx,1] + np.imag(X[n][j,i])
                X_mask[n,idx] = 0
                counter[idx] += 1

        for i in range(len(counter)):
            if counter[i] > 0:
                X_out[n,i,:] = X_out[n,i,:] / counter[i]

    return X_out, X_mask

In [ ]:
# # get labels and data seperately
truths = [s['label'] for s in data]
cm = [s['cm'] for s in data]
cm_true = [s['cm_true'] for s in data]
sensor_pos = [s['sensor_pos'][:,0] for s in data]

M = 16
coarray, coarray_mask = get_coarray(cm, sensor_pos, M)
coarray_true, _ = get_coarray(cm_true, sensor_pos, M)

In [ ]:
np.save(f"../../../data\experiment_1/scenario_1/coarray.npy", coarray)
np.save(f"../../../data\experiment_1/scenario_1/coarray_mask.npy", coarray_mask)
np.save(f"../../../data\experiment_1/scenario_1/coarray_true.npy", coarray_true)
with open(f'../../../data\experiment_1/scenario_1/truths', "wb") as fp:
    pickle.dump(truths, fp)

In [ ]:
del data
gc.collect()

In [ ]:
# select channels
X = coarray
X_mask = coarray_mask
y = coarray_true

# # standardize
# for i in range(data.shape[2]):
#     for j in range(data.shape[3]):
#         for k in range(data.shape[1]):
#             data[:,k,i,j] -= np.mean(data[:,k,i,j])
#             data[:,k,i,j] /= np.std(data[:,k,i,j])
#
# # nan to zero for diagonals
# data = np.nan_to_num(data)

In [ ]:
del cm
del cm_true
del truths
# del labels
gc.collect()

In [ ]:
# train, val split
val_ratio = 0.1
X_train, X_val, y_train, y_val, m_train, m_val = train_test_split(X, y, X_mask, test_size=val_ratio, random_state=seed)

In [ ]:
# dataloader
class FastTensorDataLoader:
    """
    A DataLoader-like object for a set of tensors that can be much faster than
    TensorDataset + DataLoader because dataloader grabs individual indices of
    the dataset and calls cat (slow).
    Source: https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
    """
    def __init__(self, *tensors, batch_size=32, shuffle=False):
        """
        Initialize a FastTensorDataLoader.
        :param *tensors: tensors to store. Must have the same length @ dim 0.
        :param batch_size: batch size to load.
        :param shuffle: if True, shuffle the data *in-place* whenever an
            iterator is created out of this object.
        :returns: A FastTensorDataLoader.
        """
        assert all(t.shape[0] == tensors[0].shape[0] for t in tensors)
        self.tensors = tensors

        self.dataset_len = self.tensors[0].shape[0]
        self.batch_size = batch_size
        self.shuffle = shuffle

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches
    def __iter__(self):
        if self.shuffle:
            r = torch.randperm(self.dataset_len)
            self.tensors = [t[r] for t in self.tensors]
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.dataset_len:
            raise StopIteration
        batch = tuple(t[self.i:self.i+self.batch_size] for t in self.tensors)
        self.i += self.batch_size
        return batch

    def __len__(self):
        return self.n_batches

In [ ]:
# dataloader, model, criterion and optimizer initialization
batch_size = 512
learning_rate = 1e-3

train_loader = FastTensorDataLoader(X_train, y_train, m_train, batch_size=batch_size, shuffle=True)
val_loader = FastTensorDataLoader(X_val, y_val, m_val, batch_size=batch_size)
model = Covariance_Reconstructer().to(device)
# model.load_state_dict(torch.load('/content/drive/MyDrive/Research/Experiments/model.pt'))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

In [ ]:
del X_train
del X_val
del y_train
del y_val
gc.collect()

In [ ]:
# training
num_epochs = 53
best_loss = 1000
lr_decay_patience = 10
lr_decay_counter = 0
train_loss_list = np.zeros(num_epochs)
train_rmse_list = np.zeros(num_epochs)
train_acc_list = np.zeros(num_epochs)
val_loss_list = np.zeros(num_epochs)
val_rmse_list = np.zeros(num_epochs)
val_acc_list = np.zeros(num_epochs)
# train_loss_list = np.load('/content/drive/MyDrive/Research/Experiments/train_loss.npy')
# train_rmse_list = np.load('/content/drive/MyDrive/Research/Experiments/train_rmse.npy')
# train_acc_list = np.load('/content/drive/MyDrive/Research/Experiments/train_acc.npy')
# val_loss_list = np.load('/content/drive/MyDrive/Research/Experiments/val_loss.npy')
# val_rmse_list = np.load('/content/drive/MyDrive/Research/Experiments/val_rmse.npy')
# val_acc_list = np.load('/content/drive/MyDrive/Research/Experiments/val_acc.npy')

for epoch in range(0,num_epochs):
    model.train()
    running_loss = 0
    counter = 0

    for batch_id, (X, y, m) in tqdm(enumerate(train_loader), leave=False, desc="training"):
        X = torch.tensor(X).float().to(device)
        y = torch.tensor(y).float().to(device)
        m = torch.tensor(m).type(torch.bool).to(device)
     
        p = model(X,m)
        loss = criterion(p[m==0], y[m==0])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.detach().data * y.shape[0]
        counter += y.shape[0]

        del X
        del y
        del p
        del loss
    gc.collect()

    train_loss = running_loss/counter
    train_loss_list[epoch] = train_loss

    torch.cuda.empty_cache()

    # validation
    with torch.no_grad():
        model.eval()
        running_loss = 0
        counter = 0

        for batch_id, (X, y, m) in tqdm(enumerate(val_loader), leave=False, desc="validation"):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            m = torch.tensor(m).type(torch.bool).to(device)

            p = model(X,m)
            loss = criterion(p[m==0], y[m==0])

            running_loss += loss.detach().data * y.shape[0]
            counter += y.shape[0]

            del X
            del y
            del p
            del loss
        gc.collect()

        val_loss = running_loss/counter
        val_loss_list[epoch] = val_loss

    print(f"epoch {epoch+1}, train-loss {train_loss:.4f}, val-loss {val_loss:.4f}")
    if val_loss < best_loss:
        best_loss = val_loss
        lr_decay_counter = 0
        torch.save(model.state_dict(), '../../../results\experiment_1/scenario_1/proposed_step1.pt')
        print(f"model is saved")
    else:
        lr_decay_counter += 1

    if lr_decay_counter == lr_decay_patience:
        lr_decay_counter = 0
        learning_rate *= 0.7
        learning_rate = max([learning_rate, 1e-7])
        print(f"learning rate decayed to {learning_rate:.4f}")

        for g in optimizer.param_groups:
            g['lr'] = learning_rate

    np.save('../../../results\experiment_1/scenario_1/train_loss_step1.npy', train_loss_list)
    np.save('../../../results\experiment_1/scenario_1/train_rmse_step1.npy', train_rmse_list)
    np.save('../../../results\experiment_1/scenario_1/train_acc_step1.npy', train_acc_list)
    np.save('../../../results\experiment_1/scenario_1/val_loss_step1.npy', val_loss_list)
    np.save('../../../results\experiment_1/scenario_1/val_rmse_step1.npy', val_rmse_list)
    np.save('../../../results\experiment_1/scenario_1/val_acc_step1.npy', val_acc_list)


In [ ]:
# loss curves
plt.figure(figsize=(10,5))
plt.plot(range(len(train_loss_list))[:53], train_loss_list[:53])
plt.plot(range(len(val_loss_list))[:53], val_loss_list[:53])
plt.title("training and validation loss curves")
plt.xlabel("epoch")
plt.ylabel("mse loss")
plt.legend(["train", "val"])
plt.grid()

In [ ]:
snr_list = [-20,-15,-10,-5,0,5,10,15,20,25,30]
T_list = [100,200,500,1000,2000,5000,10000]
cm_reconstruction_loss_list1 = []
cm_reconstruction_loss_list2 = []
cm_original_loss_list1 = []
cm_original_loss_list2 = []

for snr in snr_list:
    # load data
    data = np.load(f'../../../data\experiment_1/scenario_1/data_test_snr{snr}_t1000.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    cm = [s['cm'] for s in data]
    cm_true = [s['cm_true'] for s in data]
    sensor_pos = [s['sensor_pos'][:,0] for s in data]

    M = 16
    coarray, coarray_mask = get_coarray(cm, sensor_pos, M)
    coarray_true, _ = get_coarray(cm_true, sensor_pos, M)

    # select channels
    X = coarray
    y = coarray_true
    X_mask = coarray_mask

    # # standardize
    # for i in range(data.shape[2]):
    #     for j in range(data.shape[3]):
    #         for k in range(data.shape[1]):
    #             data[:,k,i,j] -= np.mean(data[:,k,i,j])
    #             data[:,k,i,j] /= np.std(data[:,k,i,j])
    #
    # # nan to zero for diagonals
    # data = np.nan_to_num(data)

    # dataloader, model, criterion and optimizer initialization
    batch_size = 512
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    test_loader = FastTensorDataLoader(X, y, X_mask, batch_size=batch_size)
    criterion = nn.MSELoss()

    # testing
    model = Covariance_Reconstructer().to(device)
    model.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step1.pt', map_location=torch.device('cpu'))) # map_location=torch.device('cpu')

    with torch.no_grad():
        model.eval()
        running_loss1 = 0
        running_loss2 = 0
        counter = 0

        for batch_id, (X, y, m) in enumerate(test_loader):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            m = torch.tensor(m).type(torch.bool).to(device)

            p = model(X,m)
            loss1 = criterion(torch.angle(torch.complex(p[m==0][:,0], p[m==0][:,1])), torch.angle(torch.complex(y[m==0][:,0], y[m==0][:,1])))
            loss2 = criterion(torch.angle(torch.complex(X[m==0][:,0], X[m==0][:,1])), torch.angle(torch.complex(y[m==0][:,0], y[m==0][:,1])))

            running_loss1 += loss1.item() * y.shape[0]
            running_loss2 += loss2.item() * y.shape[0]
            counter += y.shape[0]

        test_loss1 = running_loss1/counter
        test_loss2 = running_loss2/counter
        cm_reconstruction_loss_list1.append(test_loss1)
        cm_original_loss_list1.append(test_loss2)

    print(f"snr {snr}dB, T {1000}, test-loss1 {test_loss1:.4f}, test-loss2 {test_loss2:.4f}")

for T in T_list:
    # load data
    data = np.load(f'../../../data\experiment_1/scenario_1/data_test_snr-10_t{T}.npy', allow_pickle=True)

    # get labels and data seperately
    truths = [s['label'] for s in data]
    cm = [s['cm'] for s in data]
    cm_true = [s['cm_true'] for s in data]
    sensor_pos = [s['sensor_pos'][:,0] for s in data]

    M = 16
    coarray, coarray_mask = get_coarray(cm, sensor_pos, M)
    coarray_true, _ = get_coarray(cm_true, sensor_pos, M)

    # select channels
    X = coarray
    y = coarray_true
    X_mask = coarray_mask

    # # standardize
    # for i in range(data.shape[2]):
    #     for j in range(data.shape[3]):
    #         for k in range(data.shape[1]):
    #             data[:,k,i,j] -= np.mean(data[:,k,i,j])
    #             data[:,k,i,j] /= np.std(data[:,k,i,j])
    #
    # # nan to zero for diagonals
    # data = np.nan_to_num(data)

    # dataloader, model, criterion and optimizer initialization
    batch_size = 512
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    test_loader = FastTensorDataLoader(X, y, X_mask, batch_size=batch_size)
    criterion = nn.MSELoss()

    # testing
    model = Covariance_Reconstructer().to(device)
    model.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step1.pt', map_location=torch.device('cpu'))) # map_location=torch.device('cpu')

    with torch.no_grad():
        model.eval()
        running_loss1 = 0
        running_loss2 = 0
        counter = 0

        for batch_id, (X, y, m) in enumerate(test_loader):
            X = torch.tensor(X).float().to(device)
            y = torch.tensor(y).float().to(device)
            m = torch.tensor(m).type(torch.bool).to(device)

            p = model(X,m)
            loss1 = criterion(torch.angle(torch.complex(p[m==0][:,0], p[m==0][:,1])), torch.angle(torch.complex(y[m==0][:,0], y[m==0][:,1])))
            loss2 = criterion(torch.angle(torch.complex(X[m==0][:,0], X[m==0][:,1])), torch.angle(torch.complex(y[m==0][:,0], y[m==0][:,1])))

            running_loss1 += loss1.item() * y.shape[0]
            running_loss2 += loss2.item() * y.shape[0]
            counter += y.shape[0]

        test_loss1 = running_loss1/counter
        test_loss2 = running_loss2/counter
        cm_reconstruction_loss_list2.append(test_loss1)
        cm_original_loss_list2.append(test_loss2)

    print(f"snr -10dB, T {T}, test-loss1 {test_loss1:.4f}, test-loss2 {test_loss2:.4f}")

np.save('../../../results\experiment_1/scenario_1/cm_reconstruction_loss1.npy', cm_reconstruction_loss_list1)
np.save('../../../results\experiment_1/scenario_1/cm_reconstruction_loss2.npy', cm_reconstruction_loss_list2)
np.save('../../../results\experiment_1/scenario_1/cm_original_loss1.npy', cm_original_loss_list1)
np.save('../../../results\experiment_1/scenario_1/cm_original_loss2.npy', cm_original_loss_list2)

In [ ]:
def get_cov(X, pos):
    pos = (2*pos-1).astype(int)
    cov = np.zeros((len(pos), len(pos), 3))

    diff = np.zeros(X.shape[0])
    c = 0
    for i in range(len(pos)):
        for j in range(i+1,len(pos)):
            if np.sum(diff==abs(pos[i]-pos[j])) == 0:
                diff[c] = abs(pos[i]-pos[j])
                c = c+1
    diff = np.sort(diff)

    for i in range(len(pos)):
        for j in range(i+1,len(pos)):
            cov[i,j,0] = X[diff==abs(pos[i]-pos[j]),0]
            cov[i,j,1] = X[diff==abs(pos[i]-pos[j]),1]
            cov[i,j,2] = np.angle(np.vectorize(complex)(cov[i,j,0], cov[i,j,1]))
            cov[j,i,0] = X[diff==abs(pos[i]-pos[j]),0]
            cov[j,i,1] = -X[diff==abs(pos[i]-pos[j]),1]
            cov[j,i,2] = np.angle(np.vectorize(complex)(cov[j,i,0], cov[j,i,1]))

    return cov

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

fig = plt.figure(constrained_layout=True, figsize=(10.5,12))
line = plt.Line2D((.5,.5),(.01,.99), color="k", linewidth=3)
fig.add_artist(line)
line = plt.Line2D((.01,.99),(.2,.2), color="k", linewidth=3)
fig.add_artist(line)
line = plt.Line2D((.01,.99),(.4,.4), color="k", linewidth=3)
fig.add_artist(line)
line = plt.Line2D((.01,.99),(.6,.6), color="k", linewidth=3)
fig.add_artist(line)
line = plt.Line2D((.01,.99),(.8,.8), color="k", linewidth=3)
fig.add_artist(line)

subfigs = fig.subfigures(nrows=5, ncols=2, wspace=0.01)
snr_list = [-20,-10,0,10,20,30]
T_list = [100,500,2000,10000]
snr_T_list1 = [(snr,1000) for snr in snr_list]
snr_T_list2 = [(-10,T) for T in T_list]
snr_T_list = snr_T_list1 + snr_T_list2
idx = 0

for row in range(subfigs.shape[0]):
    for col in range(subfigs.shape[1]):
        snr, T = snr_T_list[idx]
        idx += 1

        # load data
        data = np.load(f'../../../data\experiment_1/scenario_1/data_test_snr{snr}_t{T}.npy', allow_pickle=True)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model = Covariance_Reconstructer().to(device)
        model.load_state_dict(torch.load('../../../results\experiment_1/scenario_1/proposed_step1.pt', map_location=torch.device('cpu')))

        # get labels and data seperately
        truths = [s['label'] for s in data]
        cm = [s['cm'] for s in data]
        cm_true = [s['cm_true'] for s in data]
        sensor_pos = [s['sensor_pos'][:,0] for s in data]

        M = 16
        coarray, coarray_mask = get_coarray(cm, sensor_pos, M)
        coarray_true, _ = get_coarray(cm_true, sensor_pos, M)

        # select channels
        X = torch.tensor(coarray).float().to(device)
        y = torch.tensor(coarray_true).float().to(device)
        X_mask = torch.tensor(coarray_mask).type(torch.bool).to(device)

        i=np.random.randint(low=0,high=len(X))
        p = model(X[i,:].unsqueeze(0), X_mask[i,:].unsqueeze(0))
        p = p.squeeze().detach().cpu().numpy()
        p = get_cov(p, sensor_pos[i])

        subfigs[row,col].suptitle(f'SNR = {snr} dB, T = {T}')

        axs = subfigs[row,col].subplots(nrows=1, ncols=3)
        axs[0].imshow(np.angle(cm[i]), vmin=-np.pi, vmax=np.pi, cmap='Greens')
        axs[0].set_title(f'Measured')
        axs[0].set_axis_off()

        axs[1].imshow(p[:,:,2], vmin=-np.pi, vmax=np.pi, cmap='Greens')
        axs[1].set_title(f'Reconstructed')
        axs[1].set_axis_off()

        axs[2].imshow(np.angle(cm_true[i]), vmin=-np.pi, vmax=np.pi, cmap='Greens')
        axs[2].set_title(f'True')
        axs[2].set_axis_off()
